In [ ]:
from pyspark.sql.functions import * 
from pyspark.sql import Window

In [ ]:
#reading all csv
Charges= spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Charges_use.csv")

Endorsements= spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Endorse_use.csv")

Restrict= spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Restrict_use.csv")

Damages= spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Damages_use.csv")

PrimaryPerson= spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Primary_Person_use.csv")

Unit = spark.read.format("csv").option("header","true").option("sep",",").option("inferSchema", "true").load("/mnt/cosmos/Units_use.csv")

In [ ]:
Unit=Unit.distinct()
PrimaryPerson=PrimaryPerson.distinct()
Damages=Damages.distinct()
Restrict=Restrict.distinct()
Endorsements=Endorsements.distinct()
Charges=Charges.distinct()

In [ ]:
Unit.count(),PrimaryPerson.count(),Damages.count(),Restrict.count(),Endorsements.count(),Charges.count()

Out[66]: (168124, 156954, 24606, 159852, 159818, 115909)

1.	Analytics 1: Find the number of crashes (accidents) in which number of persons killed are male?

In [ ]:
#Assumption : number of persons killed are male ~ gender of persons killed are male
PrimaryPerson.select("CRASH_ID","PRSN_GNDR_ID","DEATH_CNT").where((PrimaryPerson.PRSN_GNDR_ID=="MALE") & (PrimaryPerson.DEATH_CNT==1) ).distinct().count()

Out[67]: 180

Analysis 2: How many two wheelers are booked for crashes?

In [ ]:
Unit.select("VIN","VEH_BODY_STYL_ID").where((Unit.VEH_BODY_STYL_ID.contains("MOTORCYCLE")) & ~(col('VIN').isNull())).distinct().count()

Out[68]: 766

Analysis 3 Which state has highest number of accidents in which females are involved?

In [ ]:
#taking DRVR_LIC_STATE_ID for identifying states
df3=PrimaryPerson.select("CRASH_ID","DRVR_LIC_STATE_ID","PRSN_GNDR_ID").where((PrimaryPerson.PRSN_GNDR_ID=="FEMALE")).groupBy("DRVR_LIC_STATE_ID").count().orderBy(desc("count")).withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("count"))))
df3.select("DRVR_LIC_STATE_ID").filter(df3.rank==1).display()

DRVR_LIC_STATE_ID
Texas


Analysis 4: Which are the Top 5th to 15th VEH_MAKE_IDs that contribute to a largest number of injuries including death

In [ ]:
#taking sum of Unit.TOT_INJRY_CNT+Unit.DEATH_CNT for identifying number of injuries including death
df4=Unit.withColumn("TOT_INJRY_CNT_plus_DEATH_CNT",Unit.TOT_INJRY_CNT+Unit.DEATH_CNT).groupBy("VEH_MAKE_ID").agg(sum("TOT_INJRY_CNT").alias("sum_TOT_INJRY_CNT"),sum("DEATH_CNT").alias("sum_DEATH_CNT"),sum("TOT_INJRY_CNT_plus_DEATH_CNT").alias("sum_TOT_INJRY_CNT_plus_DEATH_CNT")).withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("sum_TOT_INJRY_CNT_plus_DEATH_CNT"))))
df4.select("VEH_MAKE_ID","rank").where((df4.rank>=5) & (df4.rank<=15)).display()

VEH_MAKE_ID,rank
NISSAN,5
HONDA,6
GMC,7
HYUNDAI,8
KIA,9
JEEP,10
CHRYSLER,11
MAZDA,12
NA,13
VOLKSWAGEN,14


Analysis 5: For all the body styles involved in crashes, mention the top ethnic user group of each unique body style

In [ ]:
join_unit_product=Unit.join(PrimaryPerson,on=["CRASH_ID","UNIT_NBR"],how="left")

In [ ]:
df5=join_unit_product.select("CRASH_ID","VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID","UNIT_NBR")

In [ ]:
df5.groupBy("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").count().withColumn("rank",dense_rank().over(Window.partitionBy("VEH_BODY_STYL_ID").orderBy(desc("count")))).select("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").where("rank==1").display()

VEH_BODY_STYL_ID,PRSN_ETHNICITY_ID
AMBULANCE,WHITE
BUS,BLACK
FARM EQUIPMENT,WHITE
FIRE TRUCK,WHITE
MOTORCYCLE,WHITE
NA,null
NEV-NEIGHBORHOOD ELECTRIC VEHICLE,WHITE
NOT REPORTED,WHITE
OTHER (EXPLAIN IN NARRATIVE),WHITE
"PASSENGER CAR, 2-DOOR",WHITE


Analysis 6: Among the crashed cars, what are the Top 5 Zip Codes with highest number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)

In [ ]:
#taking car as per my understanding
df6=join_unit_product.select("*").where((join_unit_product.PRSN_ALC_RSLT_ID=="Positive")&(join_unit_product.VEH_BODY_STYL_ID.isin(["VAN","NEV-NEIGHBORHOOD ELECTRIC VEHICLE","PASSENGER CAR, 2-DOOR","POLICE CAR/TRUCK","PASSENGER CAR, 4-DOOR","SPORT UTILITY VEHICLE"])))#.withColumn("rank",dense_rank().over(Window.partitionBy("VEH_BODY_STYL_ID").orderBy(desc("count"))))#.count()

In [ ]:
df6.where(~(col("DRVR_ZIP")).isNull()).groupBy("DRVR_ZIP").count().withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("count")))).select("DRVR_ZIP").where("rank<=5").display()

DRVR_ZIP
78521
79936
76010
79938
78741
79907
78233
78240


Analysis 7: Count of Distinct Crash IDs where No Damaged Property was observed and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance

In [ ]:
#taking car as per my understanding and assuming Unit.FIN_RESP_PROOF_ID==1 as car avails Insurance because no information for it was given.
df7=Unit.where((Unit.VEH_DMAG_SCL_1_ID.isin(["DAMAGED 5","DAMAGED 6","DAMAGED 7 HIGHEST"])) &(Unit.VEH_DMAG_SCL_2_ID.isin(["DAMAGED 5","DAMAGED 6","DAMAGED 7 HIGHEST"])) & (Unit.FIN_RESP_PROOF_ID==1)& (Unit.VEH_BODY_STYL_ID.isin(["VAN","NEV-NEIGHBORHOOD ELECTRIC VEHICLE","PASSENGER CAR, 2-DOOR","POLICE CAR/TRUCK","PASSENGER CAR, 4-DOOR","SPORT UTILITY VEHICLE"])))#.count()
#g.groupBy("CRASH_ID").count().filter("count>1").display()
df7.join(Damages,on=["CRASH_ID"],how="inner").distinct().count()

Out[76]: 583

Analysis 8: Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers, uses top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)

In [ ]:
#top10 Color
top10Col=Unit.distinct().groupBy("VEH_COLOR_ID").count().withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("count")))).where("rank<11")
l2= [row.VEH_COLOR_ID for row in top10Col.distinct().collect()]

In [ ]:
#top 25 states
top25state=Unit.distinct().select("VEH_LIC_STATE_ID","CRASH_ID").distinct().groupBy("VEH_LIC_STATE_ID").count().withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("count")))).where("rank<26")
l1=[row.VEH_LIC_STATE_ID for row in top25state.distinct().collect()]

In [ ]:
#where drivers are charged with speeding related offences, has licensed Drivers
DriverCharges=PrimaryPerson.select("CRASH_ID","UNIT_NBR","PRSN_NBR").where(PrimaryPerson.DRVR_LIC_TYPE_ID=="DRIVER LICENSE").join(Charges.where(lower(col("CHARGE")).contains("speed")),on=["CRASH_ID","UNIT_NBR","PRSN_NBR"]).distinct()#.display()


In [ ]:
#Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers, uses top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)
Unit.distinct().where((Unit.VEH_COLOR_ID.isin(*l2)) &(Unit.VEH_LIC_STATE_ID.isin(*l1))).join(DriverCharges.distinct(),on=["CRASH_ID","UNIT_NBR"],how="inner").groupBy("VEH_MAKE_ID").count().withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(desc("count")))).select("VEH_MAKE_ID").where("rank<6").display()

VEH_MAKE_ID
FORD
CHEVROLET
TOYOTA
DODGE
HONDA
